In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
import matplotlib.pyplot as plt

In [3]:
from numpy import loadtxt

In [4]:
import pandas as pd

In [5]:
import pydotplus

In [44]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import plot_tree

In [7]:
import xgboost as xgb
from xgboost import XGBClassifier

In [8]:
df = pd.read_excel('drug_consumption.xls')

In [9]:
demographic_features = [
    'Ethnicity',
    'Country',
    'Education',
    'Age',
    'Gender',
]

personality_traits = [
    'Nscore',
    'Escore',
    'Oscore',
    'Ascore',
    'Cscore',
    'Impulsive',
    'SS'
]

targets = [
    'Alcohol',
    'Amphet',
    'Amyl',
    'Benzos',
    'Caff',
    'Cannabis',
    'Choc',
    'Coke',
    'Crack',
    'Ecstasy',
    'Heroin',
    'Ketamine',
    'Legalh',
    'LSD',
    'Meth',
    'Shrooms',
    'Nicotine',
    'Semer',
    'VSA'
]

In [69]:
def prediction_accuracy(clf, X_train, X_test, y_train, y_test):
    # make predictions for test data
    y_pred = clf.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print(f'{clf.__class__.__name__}\n\tAccuracy: {accuracy * 100.0}')
    #data_dmat = xgb.DMatrix(data=data_frame[features], label=label_encoded_y)

In [70]:
def setup_classifier(data_frame, features, target):
    def classify(clf):
        # split data into X and y
        X = data_frame[features]
        y = data_frame[target]
    
        label_encoder = LabelEncoder()
        label_encoder = label_encoder.fit(y)
        label_encoded_y = label_encoder.transform(y)
    
        seed = 7
        test_size = 0.33
        X_train, X_test, y_train, y_test = train_test_split(X, label_encoded_y, test_size=test_size, random_state=seed)
        # fit model no training data
        clf.fit(X_train, y_train)
        return clf, X_train, X_test, y_train, y_test
    return classify

In [81]:
classifiers = [
    XGBClassifier(),
    AdaBoostClassifier(),
    RandomForestClassifier(n_estimators=10,max_depth=None,min_samples_split=2,random_state=0),
    DecisionTreeClassifier(max_depth=None,min_samples_split=2,random_state=0)
]

## Target: _Alcohol_

In [89]:
factory = setup_classifier(df, demographic_features + personality_traits, [targets[0]])

In [90]:
for clf in classifiers:
    model, X_train, X_test, y_train, y_test = factory(clf)
    prediction_accuracy(model, X_train, X_test, y_train, y_test)

/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier
	Accuracy: 37.56019261637239
AdaBoostClassifier
	Accuracy: 26.64526484751204
RandomForestClassifier
	Accuracy: 36.91813804173354
DecisionTreeClassifier
	Accuracy: 27.929373996789725


/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was 

In [109]:
factory = setup_classifier(df[df['Alcohol'] != 'CL0'], demographic_features + personality_traits, [targets[0]])

In [110]:
for clf in classifiers:
    model, X_train, X_test, y_train, y_test = factory(clf)
    prediction_accuracy(model, X_train, X_test, y_train, y_test)

/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier
	Accuracy: 36.98854337152209
AdaBoostClassifier
	Accuracy: 28.477905073649758
RandomForestClassifier
	Accuracy: 35.67921440261866
DecisionTreeClassifier
	Accuracy: 31.751227495908346


/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was 

## Target: _Meth_

In [99]:
factory = setup_classifier(df, demographic_features + personality_traits, [targets[14]])

In [100]:
for clf in classifiers:
    model, X_train, X_test, y_train, y_test = factory(clf)
    prediction_accuracy(model, X_train, X_test, y_train, y_test)

/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier
	Accuracy: 77.36757624398074
AdaBoostClassifier
	Accuracy: 33.547351524879616
RandomForestClassifier
	Accuracy: 75.76243980738363
DecisionTreeClassifier
	Accuracy: 63.7239165329053


/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was 

In [105]:
factory = setup_classifier(df[df['Meth'] != 'CL0'], demographic_features + personality_traits, [targets[14]])

In [106]:
for clf in classifiers:
    model, X_train, X_test, y_train, y_test = factory(clf)
    prediction_accuracy(model, X_train, X_test, y_train, y_test)

/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier
	Accuracy: 27.81456953642384
AdaBoostClassifier
	Accuracy: 24.503311258278146
RandomForestClassifier
	Accuracy: 24.503311258278146
DecisionTreeClassifier
	Accuracy: 17.880794701986755


/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/florian/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was 

## Future Work

- Different parametrization
- Better visualization